<a href="https://colab.research.google.com/github/brgama/tech-challenge/blob/main/TechChallenge_Fase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Carregando arquivos do Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Unsloth - biblioteca otimizada para treinar modelos de linguagem grandes (LLMs) de forma mais rápida e eficiente
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# triton - compilador da NVIDIA que otimiza operações de deep learning, acelerando o treino de modelos LLM.
!pip install triton
# Fine-tuning com LoRA, quantização e otimização de modelos
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
# Biblioteca da Hugging Face para carregar e treinar modelos como LLaMA, GPT e outros.
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-f87o3tv5/unsloth_8c4819bf3df249788736c49dd547426c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-f87o3tv5/unsloth_8c4819bf3df249788736c49dd547426c
  Resolved https://github.com/unslothai/unsloth.git to commit 2b5d81d75281c02480927cf3ca0dea7c8e98d484
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.6 kB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrt

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import json
import pandas as pd
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import pipeline

In [5]:
# Configurações do Modelo
max_seq_length = 512
dtype = None
load_in_4bit = True

In [6]:
# Modelos compatíveis
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


In [7]:
# Função para formatar o dataset

def format_dataset_into_model_input(data):
    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Alterando a formatação para título e descrição
    for example in data:
        title = example['title']
        content = example['content']
        instruction = "Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers."

        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON (caso necessário)
    formatted_json_path = "/content/drive/MyDrive/FIAP/amazon_file_formatted.json"
    with open(formatted_json_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset formatado salvo em: {formatted_json_path}")

    return formatted_json_path

In [8]:

# Caminho do arquivo JSON original
DATA_PATH = '/content/drive/MyDrive/FIAP/trn.json'

# Carregar o dataset do arquivo JSON
dataset = load_dataset("json", data_files=DATA_PATH)

# Caminho do arquivo CSV que será gerado
OUTPUT_PATH_DATASET = format_dataset_into_model_input(dataset['train'] if 'train' in dataset else dataset)

Dataset formatado salvo em: /content/drive/MyDrive/FIAP/amazon_file_formatted.json


In [9]:
# Carregando o Modelo Pré-treinado
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [10]:
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): M

In [11]:
def generate_description(title):
    input_text = f"Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.\nTítulo: {title}\nDescrição:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

In [12]:
title = "Smartphone Samsung Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone Samsung Android 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone Samsung Android 64GB
Descrição:

The Samsung Galaxy S20 FE is a powerful smartphone that offers a range of features and capabilities to enhance your mobile experience. With its 6.5-inch Super AMOLED display, you can enjoy vibrant and clear visuals for your favorite movies, games, and more. The phone is powered by a Qualcomm Snapdragon 865 processor, ensuring smooth and efficient performance.

The Galaxy S20 FE features a triple-lens camera system, including a 12MP ultra-wide lens, a 12MP wide-angle lens, and an 8MP


In [13]:
title = "Smartphone iphone 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone iphone 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone iphone 64GB
Descrição:

The iPhone 64GB is a powerful and versatile smartphone that offers a range of features and capabilities to enhance your daily life. With its sleek design, advanced technology, and intuitive user interface, the iPhone 64GB is a top choice for those who demand the best in mobile technology.

One of the standout features of the iPhone 64GB is its large, high-resolution display. The 6.1-inch Super Retina XDR display offers stunning visuals with a resolution of 2532 x 1170 pixels, making it perfect for


In [14]:
# Ajustes finos do LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # Aumentado para melhorar a capacidade de ajuste
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,  # Aumentado para ajustar melhor os parâmetros
    lora_dropout=0.1,  # Introduzido dropout para evitar overfitting
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [15]:
# Prompt para a descrição dos Produtos
alpaca_prompt = """Below is an instruction outlining the task, accompanied by a title that provides additional context. Write a response that appropriately concludes the description.

### Instrução:
{}

### Título:
{}

### Descrição:
{}"""

EOS_TOKEN = tokenizer.eos_token

In [16]:
# Função para formatar o Prompt

def formatting_prompts_func(data):
    instructions = data["instruction"]
    inputs = data["input"]
    outputs = data["output"]

    texts = []

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

In [17]:
# Carregar o dataset
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET)

print(dataset)
print(dataset['train'].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 2248619
    })
})
['instruction', 'input', 'output']


In [18]:
# Verificar uma amostra
print(dataset['train'][0])  # Verifique uma amostra do dataset

{'instruction': 'Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.', 'input': 'Girls Ballet Tutu Neon Pink', 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}


In [19]:
formatted_dataset = dataset['train'].map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/2248619 [00:00<?, ? examples/s]

In [20]:
formatted_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 2248619
})

In [21]:
dataset['train'][0]

{'instruction': 'Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.',
 'input': 'Girls Ballet Tutu Neon Pink',
 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}

In [22]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,  # Aumento dos warmup steps para melhorar a estabilidade do treinamento
        max_steps=60,  # Aumento no número de passos de treinamento
        learning_rate=1e-4,  # Taxa de aprendizado ajustada para um ajuste mais refinado
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,  # Log mais frequente para monitorar o treinamento
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/2248619 [00:00<?, ? examples/s]

In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,248,619 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080/3,842,248,704 (2.18% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: brunogamarj (brunogamarj-yduqs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.408200
20,1.892100
30,1.317400
40,1.279200
50,1.349700
60,1.232700


In [26]:
# Salvar o modelo ajustado
model.save_pretrained("/content/drive/MyDrive/FIAP/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/FIAP/lora_model")

('/content/drive/MyDrive/FIAP/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/lora_model/tokenizer.model',
 '/content/drive/MyDrive/FIAP/lora_model/added_tokens.json',
 '/content/drive/MyDrive/FIAP/lora_model/tokenizer.json')

In [27]:

# Caminho para o modelo salvo
model_path = "/content/drive/MyDrive/FIAP/lora_model"

# Carregar o modelo e o tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(model_path)

# Preparar o modelo para inferência
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/FIAP/lora_model as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
    

In [28]:
title = "Smartphone Samsung Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone Samsung Android 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone Samsung Android 64GB
Descrição:

### Product Description

Samsung Galaxy S6 Edge 64GB

### From the Manufacturer

##### Samsung Galaxy S6 Edge 64GB

The Samsung Galaxy S6 Edge is a smartphone that is designed to be a work of art. The Galaxy S6 Edge is the first smartphone to feature an edge-to-edge display, which is curved on both sides. The curved display is not only beautiful, but it also provides a more comfortable grip and easier access to the phone's features. The Galaxy S6 Edge is also the first smartphone to feature


In [29]:
title = "Smartphone iphone 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone iphone 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone iphone 64GB
Descrição:

### Product Description

iPhone 6 Plus is an elegant, advanced smartphone that's thinner and lighter than iPhone 5s. It features a 5.5-inch Retina HD display with 1920-by-1080 resolution, 1.4 million pixels, and 401 ppi. iPhone 6 Plus is powered by the A8 chip with second-generation 64-bit desktop-class architecture, and an M8 motion coprocessor. It features a 8MP iSight camera with 1.5µ pixels
